In [26]:
import multiprocess as mp
from functools import *

class Heightmap:
    def __init__(self, input:str):
        row = 0
        self.map= []
        for r in input.splitlines():
            rowOutput = []
            col = 0
            for c in r:
                if c == 'S':
                    self.startPos = (row, col)
                    height = 1
                elif c == 'E':
                    self.endPos = (row, col)
                    height = 26
                else:
                    height = ord(c) - ord('a') + 1
                rowOutput.append(height)
                col += 1
            self.map.append(rowOutput)
            row += 1
        #set boundaries of map
        self.height = len(self.map)
        self.width = len(self.map[0])


def shortestPath(map:Heightmap, withStartPosition=None)->int:
    #adjustable start posiiton
    if withStartPosition==None:
        withStartPosition = map.startPos

    
    #calculate the shortest path between S and E
    #can only step up 1 height or down any height -> our graph of valid moves from from the height
    infiniteNumber = 1000000 #a very large number (approx infinity)
    distanceMap:{tuple[int, int], int} = {} #the shortest distance from the start to each position
    possibleMoves = [(0,1),(0,-1),(1,0),(-1,0)] #all possible moves

    unvisited:set(tuple[int,int]) = set()
    for r in range(map.height):
        for c in range(map.width):
            unvisited.add((r,c))
            distanceMap[(r,c)] = infiniteNumber

    distanceMap[withStartPosition] = 0 #zero steps to get to the start
    currentNode = withStartPosition
    currentDistance = 0

    while len(unvisited) > 0:
        #NEED TO SELECT CURRENT NODE - smallest distance of unvisited nodes
        minDistance = infiniteNumber #a very large number (approx infinity)
        currentNode = None
        for (k, v) in distanceMap.items():
            if v <= minDistance and k in unvisited:
                minDistance = v
                currentNode = k

        if currentNode == None:
            raise Exception('Something has gone terribly wrong')

        #get info about current node
        currentHeight = map.map[currentNode[0]][currentNode[1]]
        currentDistance = minDistance
        targetDistance = currentDistance + 1
        for move in possibleMoves:
            targetY = currentNode[0] + move[0]
            targetX = currentNode[1] + move[1]
            if targetX >= 0 and targetX < map.width and targetY >= 0 and targetY < map.height:
                #in the board of play
                target = (targetY, targetX)
                #skip if we've already fully visited this node
                if target in unvisited:
                    targetH = map.map[targetY][targetX]
                    if targetH <= currentHeight + 1:
                        #valid move
                        if target in distanceMap:
                            currentShortestToTarget = distanceMap[target]
                            distanceMap[target] = min(currentShortestToTarget, targetDistance)
                        else:
                            distanceMap[target] = targetDistance
        #we're fully done with current node
        unvisited.remove(currentNode)
        #check if we're done
        if currentNode == map.endPos:
            #we're done!
            break
    return distanceMap[map.endPos]

def part2shortestPath(map)->int:
    #iterate through all the candidate heights that are 1
    pathLengths = [] #store all the possible path lengths - we will then want shortest
    startPositions:[tuple[int,int]] = []
    for r in range(map.height):
        for c in range(map.width):
            posHeight = map.map[r][c]
            if posHeight == 1:
                startPositions.append((r,c))
    print('Possible start locations: '+ str(len(startPositions)))
    print('Let\'s hack time... using ' + str(mp.cpu_count())+ ' CPUs')
    pool = mp.Pool(mp.cpu_count())
    partialSP = partial(shortestPath, map)
    pathLengths = pool.map(partialSP, startPositions)
    pool.close()
    
    pathLengths.sort()
    return pathLengths[0]




In [27]:
input = open("day12input.txt").read()
hm = Heightmap(input)
print(part2shortestPath(hm))

Possible start locations: 1779
Let's hack time... using 10 CPUs
480


Interstingly, the parallelised version of this took the same amount of time as single threaded... not sure that quite worked as intended!
pool.apply is the wrong approach, should be using map
